In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as Image
from tensorflow import keras
import os
from skimage.io import imread, imshow
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Add, Dense, Activation, Flatten
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint


### Loading images dataset


In [ ]:
# get the path of the pseudo labelled images, and using them as a pretext training task
trainpath= ('E/Dataset/data-augmentation')

In [ ]:
img_height=224
img_width=224
batch_size=128

train_datagen = ImageDataGenerator(rescale=1./255,
    validation_split=0.2) # set validation split

print("The data is being split into training and validation set")

train_generator = train_datagen.flow_from_directory( trainpath,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data
print("----------------------------------------------------------------")

validation_generator = train_datagen.flow_from_directory(trainpath,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

#define labels for training
class_names = train_generator.class_indices
print('Number of Pseudo-Labels after applying DSCAN algorithm: ',class_names)

### Building the model


In [ ]:
# Load the pretrained model without the output of the last convolution block 

base_model = ResNet50(include_top=False, input_shape=(224, 224, 3), weights = 'imagenet')

# Flatten the output layer to 1 dimension
x = Flatten()(base_model.output)

# Add a fully connected layer with 2048 hidden units and ReLU activation
x = Dense(1024, activation='relu')(x)

# Adapt the final classification layer of an ImageNet pre-trained cNN model to the decomposed classes.
predictions = Dense(len(class_decomposition), activation='softmax')(x)

base_model = Model(inputs=base_model.input, outputs=predictions)

base_model.summary()    

### Training the model 


In [ ]:
#define checkpoint
checkpoint = ModelCheckpoint(filepath='pretext_braintumour.hdf5', 
                             monitor='val_accuracy',
                             save_best_only=True, 
                             mode='auto',
                             verbose=1)   

#early stopping
earlystop=EarlyStopping( monitor="val_accuracy", 
                        patience=10,  
                        mode="auto")

model_callbacks = [checkpoint, earlystop]


optimize = SGD(learning_rate=0.0001, decay=0.9 / 10, momentum=0.9, nesterov=True)
DeTraC_model = Model(inputs=base_model.input, outputs=predictions)

#compile model
DeTraC_model.compile(loss='mse', optimizer=optimize, metrics=['accuracy']) 
    
# train the model
history=DeTraC_model.fit(train_generator, 
                               steps_per_epoch=train_generator.n//train_generator.batch_size,
                               validation_data= validation_generator,
                               validation_steps=validation_generator.n//validation_generator.batch_size,
                               epochs=50, 
                               callbacks= model_callbacks, 
                               verbose=1, shuffle= True)


In [ ]:
#showing results and model accuracy 
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)

plt.figure(figsize=(12, 6))
plt.grid()

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### Evaluate the model performance

In [ ]:
val_Loss, val_Acc = pretext_model.evaluate(validation_generator)